In [1]:
import numpy as np

In [2]:
# Define Constants. Work in SI units only to avoid excessive consersion.
rxtr_diameter = 0.1 # fixed bed diameter of 10 cm
rxtr_cross_section_area = np.pi*rxtr_diameter**2/4
volume_void_fraction = 0.4 # assume packing efficiency is 0.6 for most packed beds
area_void_fraction = 0.3 # derived from the volume void fraction
effective_area = rxtr_cross_section_area*area_void_fraction
R = 8.314e-5 # gas constant in m^3-bar/K-mol
R_2 = 8.314 # gas constant in J/mol-K or m^3-Pa/K-mol or kg-m^2/(K-mol-s^2)
g = 9.81 # gravitational acceleration in m/s^2

"""
Assume 4 HAN -> 3 N2O + 2 HNO3 +7H2O
species 0, 1, 2, 3 = HAN, N2O, HNO3, H2O
"""
MW = np.array([96, 44, 63, 18])
Shomate = ((74.826, 0, 0, 0, 0),
           (27.680, 51.145, -30.645, 6.8479, -0.15791), 
           (19.632, 153.96, -115.84, 32.880, -0.24911), 
           (30.092, 6.8325, 6.7934, -2.5345, 0.082139)) # HAN, N2O, HNO3, H2O

In [3]:
def f_Cp(i, T):
    # returns Cp in J/mol-K, i is the species
    A, B, C, D, E = Shomate[i]
    t = T/1000
    return A+B*t+C*t*t+D*t*t*t+E/t/t

In [4]:
def f_Gamma(i, T):
    # returns specific heat ratio, i is the species
    Cp = f_Cp(i, T)
    return Cp/(Cp-R_2)

In [5]:
def f_concentration(pressure, temperature):
    """
    returns concentration in mol/m^3, assuming ideal gas
    pressure in bar
    temperature in K
    """
    return pressure/(R*temperature)

In [6]:
def f_density(molecular_weight, concentration):
    """
    returns density in kg/m^3, assuming ideal gas
    molecular_weight in g/mol
    """
    return molecular_weight*concentration/1000

In [7]:
def f_effective_velocity(flow_rate, density):
    """
    returns effective velocity in m/s
    flow_rate in kg/s
    density in kg/m^3
    effective_area in m^2
    """
    return flow_rate/(density*effective_area)

In [8]:
def f_reynolds(density, effective_velocity, particle_diameter, dynamic_viscosity):
    """
    returns Re in a packed bed
    density in kg/m^3
    effective_velocity in m/s
    particle_diameter in m
    dynamic_viscosity in Pa.s
    don't know if this is useful yet
    """
    return density*effective_velocity*particle_diameter/dynamic_viscosity

In [9]:
def f_MW_mixture(x):
    """
    returns average molecular weight of mixture
    x is a numpy array containing mole fraction of each of the four species
    """
    return MW*x

In [10]:
def f_gamma_mixture(gamma, x):
    """
    returns average specific heat ratio of mixture
    gamma is a numpy array containing specific heat ratio of each of the four species
    x is a numpy array containing mole fraction of each of the four species
    """
    return gamma*x

In [11]:
def rate_law_HAN(temperature, conc_HAN):
    """
    returns rate law of HAN in mol/(m^3-s)
    temperature in K
    conc_HAN in mol/m^3
    data from https://pubs-acs-org.ezproxy.neu.edu/doi/full/10.1021/acs.jpca.8b05351
    """
    rate_constant = np.exp(21.1)*np.exp(-88.6*1000/(R_2*temperature))
    return -rate_constant*conc_HAN

In [12]:
def f_dPdZ(m, mu, rho, dp):
    """
    returns pressure drop in bar/m
    m is mass flow rate in kg/s
    G is bed loading factor in kg/(m^2-s)
    rho is average density in kg/m^3
    mu is viscosity in Pa-s
    dp is particle diameter in m
    """
    rp = dp/2
    G = m/effective_area
    phi = volume_void_fraction
    t1 = 300*(1-phi)*mu/rp+1.75*G
    t2 = -2*G/(rho*rp)*(1-phi)/(phi^3)
    return t1*t2/1e5

In [13]:
def f_dTdZ(r, dH, N, Cp):
    """
    returns dT/dZ
    r is reaction rate in mol/(m^3-s)
    dH is enthalpy of reaction for 1 mole of HAN reacted in J/mol
    N is a numpy array containing molar flow rates of all species in mol/s
    Cp is a numpy array containing specific heat of all species in J/mol-K
    """
    t1 = r*dH
    t2 = sum(N*Cp)
    return t1/t2*effective_area

In [14]:
def f_throat_area(flow_rate, chamber_pressure, chamber_temperature, gamma, MW):
    """
    returns throat area in m^2
    flow rate in kg/s
    chamber temperature in K
    chamber pressure in bar
    MW in g/mol
    """
    temp_R = R_2/MW*1000
    temp_var1 = np.sqrt(gamma/temp_R)/((gamma+1)/2)**((gamma+1)/(2*(gamma-1)))
    return flow_rate*np.sqrt(chamber_temperature)/(chamber_pressure*temp_var1*1e5)

In [15]:
# Need to write a function that solves for the Mach number given the expansion ratio
def f_Mach_number(something):
    return something

In [16]:
def f_speed_of_sound(MW, gamma, T):
    """
    returns speed of sound in m/s
    R is gas constant in kg-m^2/(K-mol)
    MW is average molecular weight in g/mol
    gamma is average specific heat ratio
    T is temperature in K
    """
    return np.sqrt(gamma*R_2*T*1000/MW)

In [17]:
def f_exhaust_velocity(Mach_number, speed_of_sound):
    "returns exhaust velocity in m/s"
    return Mach_number*speed_of_sound

In [18]:
def helper_function_1(Mach_number, gamma):
    "A helper function to calculate exhaust temperature and pressure"
    return 1+(gamma-1)/2*Mach_number**2

In [19]:
def f_exhaust_temperature(chamber_temperature, gamma, Mach_number):
    "returns exhaust temperature in K"
    return chamber_temperature/helper_function_1(Mach_number, gamma)

In [20]:
def f_exhaust_pressure(chamber_pressure, gamma, Mach_number):
    "returns exhaust pressure in bar"
    return chamber_pressure/(helper_function_1(Mach_number, gamma)**(gamma/(gamma-1)))

In [21]:
def f_thrust(Ue, flow_rate, nozzle_area, exhaust_pressure, ambient_pressure):
    """
    returns thrust in N
    Ue is exhaust velocity in m/s
    flow rate in kg/s
    nozzle area in m^2
    exhaust and ambient pressure in Pa
    """
    return Ue*flow_rate+nozzle_area*(exhaust_pressure-ambient_pressure)

In [22]:
def f_Isp(thrust, flow_rate):
    """
    returns specific impulse in s
    thrust in N
    Ue is exhaust velocity in m/s
    """
    return thrust/(flow_rate*g)

In [23]:
# Define range of variable parameters
particle_diameter_range = (0.001, 0.01) # 1-10 mm
bed_length_range = (0.1, 0.5) # range of bed length 10-50 cm
flow_rate_range = (0.001, 0.1) # 1-100 g/s
start_temperatre_range = (323, 473) # 50-200 C
feed_pressure_range = (5, 50)
water_mole_fraction_range = (0, 0.4) # range of H2O mole fraction
expansion_ratio_range = (10, 50) # range of nozzle expansion ratio

In [24]:
# This part is the core of our sensitivity analysts, needs to be discussed on Tuesday

# assign some values first to see how the model works
bed_length = 0.1
particle_diameter = 0.01
flow_rate = 0.01
start_temperature = 473
feed_pressure = 20
water_mole_fraction = 0
expansion_ratio = 50
step_size = bed_length/1000

#